In [1]:
import torch
#import Trainer
#from network import NFM
import torch.utils.data as Data
#from Utils.criteo_loader import getTestData, getTrainData

nfm_config = \
{
    'n_class':9,
    'linear_hidden1':2000,
    #'linear_hidden':100,#线性模型输出层（隐层个数）
    #'embed_input_dim':1001,#embed输入维度
    #'embed_dim': 100, # 用于控制稀疏特征经过Embedding层后的稠密特征大小，embed输出维度
    #'dnn_hidden_units': [100,11],#MLP隐层和输出层
    
    'dnn_hidden_units':[100,8],#MLP隐层
    'num_sparse_features_cols':10477,#the number of the gene columns
    'num_dense_features': 0,#dense features number
    'bi_dropout': 0.5,#Bi-Interaction 的dropout
    'num_epoch': 500,#训练epoch次数
    'batch_size': 1,#batch_size
    'lr': 1e-3,
    'l2_regularization': 1e-4,
    'device_id': 0,
    'use_cuda': False,
    'epoch':1000,
    
    #'train_file': '../Data/criteo/processed_data/train_set.csv',
    #'fea_file': '../Data/criteo/processed_data/fea_col.npy',
    #'validate_file': '../Data/criteo/processed_data/val_set.csv',
    #'test_file': '../Data/criteo/processed_data/test_set.csv',
    #'model_name': '../TrainedModels/NFM.model'
    #'train_file':'data/xiaoqiu_gene_5000/train/final_5000_encode_100x.csv',
    #'train_data':'dataset/qiuguan/encode/encode_1000/train/train_encode_data_1000_new.csv',
    #'train_label':'dataset/qiuguan/non_code/train/train_label.csv',
    #'guan_test_data':'dataset/qiuguan/non_code/guan_test/guan_test_data.csv',
    #'guan_test_label':'dataset/qiuguan/non_code/guan_test/guan_test_label.csv',
    #'test_data':'dataset/qiuguan/encode/encode_1000/test/test_encode_data_1000_new.csv',
    #'test_label':'dataset/qiuguan/non_code/test/test_labels.csv',
    #'title':'dataset/xiaoguan/RF/RF_for_train/train_class_9/test/test_data.csv',
    'gene_name':'dataset/qiuguan/origin_800/gene_name.csv',
    'label_name':'dataset/qiuguan/origin_800/gene_label.csv'
    #'all':''
    #'title':'data/xiaoqiu_gene_5000/train/gene_5000_gene_name.csv',
    #'all':'data/xiaoqiu_gene_5000/train/gene_5000_label_name.csv'
}
#model definition
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model = MLP().cuda()
print(model)

import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
#import config
#import evaluate
#import data_utils
#import Trainer

def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


import pandas as pd 

from sklearn.model_selection import train_test_split




def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output

class KZDatasetTest(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, csv_path):
    
        self.data_info = self.get_data_info(csv_path)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
    
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',',header=None)
        
        df=df.iloc[1:,1:]
        #print("df:",df)
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(i))
        #print('labels:',labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #
        
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            #print("i,data:",i,data)
            #data=pd.DataFrame(data,dtype=float)###############
            data=np.array(data)##
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
           
            
            data_info.append((data,label))
        return data_info
    
    
    
    
import torch
import torch.nn as nn
from torch.utils.data.dataset import *
from PIL import Image
from torch.nn import functional as F
import random
from sklearn.model_selection import train_test_split
import ast
import torchvision



#######找特征基因#############从3301中找200个基因
#########################################################本次测试的目的是看200个基因的分类效果
##########测试步骤：从3301个基因中提取350个
############用200个构建新的分类模型
#################特征基因
######################为小球，根据上边的测试的基因个数，350最大
import torch

import torch.nn as nn
import torch.optim as optim


from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#from mnist_test import Net, train, test


# Network parameters
class Params(object):
    batch_size = 64
    test_batch_size = 20
    epochs = 5
    lr = 0.01
    momentum = 0.5
    no_cuda = True
    seed = 1
    log_interval = 10
    
    def __init__(self):
        pass

args = Params()
torch.manual_seed(args.seed)
#device = torch.device("cpu")
device=torch.device('cuda')
kwargs = {}



##############数据准备
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
#功能：加载保存到path中的各层参数到神经网络
#path='dataset/qiuguan/model_new_K_fold_RandomTree/MLP_non_encode/MLP9110.pkl'
#path='dataset/qiuguan/origin_800/non_encode_aug/para0.4_0.6_0/MLP10210.pkl'
path='dataset/qiuguan/origin_800/non_encode_aug/aug_with_simple/para_gause_0.8_0.6/MLP10710.pkl'

#path='dataset/qiuguan/origin_800/non_encode_aug/aug_model/para_gause_0.4_0.2_0.6_0.2/MLP7910.pkl'
#path='dataset/qiuguan/origin_800/non_encode_aug/aug_et/para_gause_0.8_0.5_gause_0.3_0.1/MLP11012345.pkl'
#nfm=NFM(nfm_config)
mlp=MLP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
mlp.load_state_dict(torch.load(path),strict=False)
mlp.cuda()

print(mlp)




mlp_params = list(mlp.named_parameters())
#print(nfm_params)
net=mlp
model=net###########
'''
testset = KZDatasetTest(csv_path='../NFM-pyorch-master/dataset/qiuguan/orign/')
   
test_loader = DataLoader(
         dataset=testset,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size']
        
     )
'''

testset_xiaoqiu  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoqiu/test_info.csv')#样本收集特征数据集，和测试数据集不同，这里边可能还包含训练集
   
test_loader_xiaoqiu = DataLoader(
         dataset=testset_xiaoqiu,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )

testset_xiaoguan  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_xiaoguan = DataLoader(
         dataset=testset_xiaoguan,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )
import pandas as pd 
import torch
import numpy as np
test_df=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu/test_info.csv',sep=',')
test_df=test_df.iloc[:,1:]
rows,cols=test_df.shape
print(rows,cols)
columns=test_df.columns[:-1].tolist()
# 将二维列表转换为一维列表
#flattened_indices = [item for sublist in top_k_indices for item in sublist]

#LRP
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator2 import InnvestigateModel
#from utils import Flatten
model.double()
model.eval()
model.cuda()

inn_model = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)

genes_features=np.array([i for i  in range(9)])#################
genes_features=genes_features.reshape(9,1).tolist()#######################genes_features[i][0]=label

inn_model.cuda()

###[3, 25.353026075712233, tensor([ 182,  879,  103, 2657, 2489,  914, 2437,  180, 2417, 1402, 2344, 2947,
###values=tensor([0.0572, 0.0495, 0.0404, 0.0381, 0.0364, 0.0353, 0.0328, 0.0302, 0.0284,
#        0.0257, 0.0180, 0.0164, 0.0155, 0.0150, 0.0135, 0.0127, 0.0121, 0.0120,
#        0.0114, 0.0107, 0.0107, 0.0104, 0.0100, 0.0100, 0.0099, 0.0091, 0.0091,
#       0.0088, 0.0083, 0.0082, 0.0081, 0.0079, 0.0078, 0.0077, 0.0076, 0.0074,
#       0.0073, 0.0072, 0.0072, 0.0071], dtype=torch.float64),
###indices=tensor([ 182,  879,  103, 2657, 2489,  914, 2437,  180, 2417, 1402, 2344, 2947,
#        2546, 1114,  796, 1111, 2472, 2326, 1274,  932, 2476,  716,  989, 3289,
#        1252, 2053,  785, 2429, 3015, 1585,  975, 1150, 1155,  726,  823, 2303,
#         699,  349, 1792, 1524])), 25.359040192320702, tensor([2489,  826, 1753, 1792,  601, 2303, 1053,  545, 2559,  624, 3256,  762,
#        2666,  182, 1881, 1585,  726, 1367, 2405, 1171, 2947, 2093,   14,  265,
#         716,  180, 1467, 2207, 3223,  349,  277, 2141, 2878, 2427, 2326, 1111,
#         746, 1402, 2150, 1602]), torch.return_types.topk(
#values=tensor([0.0456, 0.0445, 0.0406, 0.0349, 0.0277, 0.0273, 0.0246, 0.0236, 0.0225,
#        0.0221, 0.0213, 0.0200, 0.0185, 0.0182, 0.0181, 0.0153, 0.0151, 0.0142,
##        0.0141, 0.0140, 0.0138, 0.0123, 0.0121, 0.0115, 0.0109, 0.0108, 0.0107,
#        0.0102, 0.0102, 0.0098, 0.0094, 0.0093, 0.0092, 0.0089, 0.0082, 0.0081,
#        0.0080, 0.0080, 0.0079, 0.0076], dtype=torch.float64),
#indices=tensor([2489,  826, 1753, 1792,  601, 2303, 1053,  545, 2559,  624, 3256,  762,
#        2666,  182, 1881, 1585,  726, 1367, 2405, 1171, 2947, 2093,   14,  265,
#         716,  180, 1467, 2207, 3223,  349,  277, 2141, 2878, 2427, 2326, 1111,
#         746, 1402, 2150, 1602]))]

for data, target in test_loader_xiaoqiu:############小球

    data, target = data.to(device), target.to(device)
    #targets=torch.max(targets,1)[1]###################
    print('data:',data.shape)
    batch_size = int(data.size()[0])
    #print('batch_size:',batch_size)#=20
    evidence_for_class = []
    #print("target:",target.shape)
    #print('target:',target[3])
    # Overlay with noise 
    # data[0] += 0.25 * data[0].max() * torch.Tensor(np.random.randn(28*28).reshape(1, 28, 28))
    #model_prediction, true_relevance = inn_model.innvestigate(in_tensor=data)
    contri_k=[]
    contri_k_names=[]
    for i in range(9):#10类
    # Unfortunately, we had some issue with freeing pytorch memory, therefore 
    # we need to reevaluate the model separately for every class.
        model_prediction, input_relevance_values = inn_model.innvestigate(in_tensor=data)
        evidence_for_class.append(input_relevance_values)
        top_k=torch.topk(input_relevance_values,20,largest=True)#################20
        print('top_k:',top_k)
    
        contri_k.append(top_k)
        top_k_indices=top_k.indices.cpu().detach().numpy().tolist()
        print('top_k_indices:',top_k_indices)
        # 将二维列表转换为一维列表
        flattened_indices = [item for sublist in top_k_indices for item in sublist]

        top_k_names=[columns[j] for j in flattened_indices]
        print('top_k_names:',top_k_names)
        contri_k_names.append(top_k_names)
        print('contri_k_names:',contri_k_names)
        
        
        continue
    #print('input_relevance_values:',input_relevance_values.shape)
    #print('evidence_for_class:',len(evidence_for_class))
    evidence_for_class = np.array([elt.numpy() for elt in evidence_for_class])
    #print('evidence_for_class:',evidence_for_class.shape)#[10,20,784]
    
    
    for idx, example in enumerate(data):#batch 中的每一个样本
        #print('example:',example.shape)
        prediction = np.argmax(model_prediction.cpu().detach(), axis=1)#
        #print('prediction[idx]:',prediction[idx])
        #print(evidence_for_class[prediction[idx]][idx])
        #fig, axes = plt.subplots(3, 5)
        '''
        fig.suptitle("Prediction of model: " + str(prediction[idx]) + "({0:.2f})".format(
            100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())))
        '''
        prediction_value=prediction[idx]
        p_x=model_prediction[idx][model_prediction[idx].argmax()].exp()
        p_sum=model_prediction[idx].exp().sum()
        prediction_score=100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())
        print('prediction_value:',prediction_value)
        print('prediction_score:',prediction_score)

        continue
        #print('分子:',p_x)
        #print('分母：',p_sum)
        #uu=pr
        #print("torch.argmax:",torch.argmax(target[idx]))
        if len(genes_features[prediction_value])==1:#有值，但还没有添加预测分数和特征值，只有标签#prediction_value代表第几种疾病
            if prediction_value!=torch.argmax(target[idx]).cpu().detach()  :
                print('不合格****************:',prediction_value)
            if prediction_value==torch.argmax(target[idx]).cpu().detach()  :#预测正确
                genes_features[prediction_value].append(prediction_score)
                print('合格：',prediction_value)
                relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                index=torch.topk(relevance_score_for_every_pixel,10,largest=True)#基因个数50#####150
                #print('pixel_sorted:',index)
                genes_features[prediction_value].append(index.indices)#添加前50基因特征
                genes_features[prediction_value].append(index)
        else: 
            if genes_features[prediction_value][1]<prediction_score:#如果值比已有值大，说明预测更准确
                if prediction_value==torch.argmax(target[idx]).cpu().detach():
                    print('&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&:',prediction_value)
                    genes_features[prediction_value].pop(3)#删除index
                    genes_features[prediction_value].pop(2)#先删除特征值
                    genes_features[prediction_value].pop(1)#先删除预测分数
                
                    genes_features[prediction_value].append(prediction_score)
        
                    relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                    #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                    relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                    index=torch.topk(relevance_score_for_every_pixel,10,largest=True)#基因个数50  200
                    #print('pixel_sorted:',index)
                    genes_features[prediction_value].append(index.indices)
                    genes_features[prediction_value].append(index)
print('qiu_____genes_features.shape:',genes_features)#######找特征基因#############从3301中找200个基因









MLP(
  (bn0): BatchNorm1d(3300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3300, out_features=2000, bias=True)
  (bn1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
MLP(
  (bn0): BatchNorm1d(3300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3300, out_features=2000, bias=True)
  (bn1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

top_k: torch.return_types.topk(
values=tensor([[0.0583, 0.0577, 0.0366, 0.0366, 0.0361, 0.0337, 0.0283, 0.0283, 0.0274,
         0.0274, 0.0272, 0.0203, 0.0202, 0.0202, 0.0199, 0.0191, 0.0190, 0.0146,
         0.0146, 0.0130]], dtype=torch.float64),
indices=tensor([[1333, 2344, 2141, 1609,  572,  826, 1171, 2332, 1426, 1053, 1209, 1270,
          386,  823, 2066,  487, 1154, 1753, 2472, 2437]]))
top_k_indices: [[1333, 2344, 2141, 1609, 572, 826, 1171, 2332, 1426, 1053, 1209, 1270, 386, 823, 2066, 487, 1154, 1753, 2472, 2437]]
top_k_names: ['LYPLA1', 'VTCN1', 'TFAP2A', 'CTSG', 'ACKR1', 'RHOBTB1', 'ZNF195', 'TESC', 'WASHC3', 'NPIPB15', 'GPX2', 'CFB', 'DNASE1L3', 'CHP1', 'BDKRB2', 'AADAC', 'TRPV6', 'DEPTOR', 'PDE3B', 'NUDT4']
contri_k_names: [['LYPLA1', 'VTCN1', 'TFAP2A', 'CTSG', 'ACKR1', 'RHOBTB1', 'ZNF195', 'TESC', 'WASHC3', 'NPIPB15', 'GPX2', 'CFB', 'DNASE1L3', 'CHP1', 'BDKRB2', 'AADAC', 'TRPV6', 'DEPTOR', 'PDE3B', 'NUDT4'], ['LYPLA1', 'VTCN1', 'TFAP2A', 'CTSG', 'ACKR1', 'RHOBTB1', 'ZN

top_k: torch.return_types.topk(
values=tensor([[0.0517, 0.0489, 0.0486, 0.0418, 0.0382, 0.0336, 0.0296, 0.0275, 0.0252,
         0.0248, 0.0236, 0.0234, 0.0218, 0.0208, 0.0197, 0.0167, 0.0166, 0.0136,
         0.0119, 0.0116]], dtype=torch.float64),
indices=tensor([[1818, 3288,  180, 1781, 1155, 1792,  879,   14, 1367, 2408,  826,  417,
         2437,  785,  601, 1426,  877,  510, 1252, 1848]]))
top_k_indices: [[1818, 3288, 180, 1781, 1155, 1792, 879, 14, 1367, 2408, 826, 417, 2437, 785, 601, 1426, 877, 510, 1252, 1848]]
top_k_names: ['ZNHIT1', 'RGS5', 'AUTS2', 'KPNA3', 'RGN', 'C1orf68', 'RHOB', 'FKBP4', 'NDUFB1', 'AMDHD2', 'RHOBTB1', 'RRM2', 'NUDT4', 'AKR1C2', 'ACTA2', 'WASHC3', 'PF4', 'ABCB7', 'FOS', 'PEX6']
contri_k_names: [['ZNHIT1', 'RGS5', 'AUTS2', 'KPNA3', 'RGN', 'C1orf68', 'RHOB', 'FKBP4', 'NDUFB1', 'AMDHD2', 'RHOBTB1', 'RRM2', 'NUDT4', 'AKR1C2', 'ACTA2', 'WASHC3', 'PF4', 'ABCB7', 'FOS', 'PEX6'], ['ZNHIT1', 'RGS5', 'AUTS2', 'KPNA3', 'RGN', 'C1orf68', 'RHOB', 'FKBP4', 'NDUFB1', 

rel_for_clas=None: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000]],
       device='cuda:0', dtype=torch.float64)
top_k: torch.return_types.topk(
values=tensor([[0.1237, 0.0646, 0.0436, 0.0384, 0.0329, 0.0271, 0.0236, 0.0235, 0.0168,
         0.0159, 0.0158, 0.0157, 0.0154, 0.0151, 0.0140, 0.0130, 0.0115, 0.0114,
         0.0113, 0.0108]], dtype=torch.float64),
indices=tensor([[1053,  487, 2405, 1333,  233, 2438, 1224,  673, 1792,  277,  103, 3282,
         2895, 1778,  823,   14,   89, 3015, 3069, 1585]]))
top_k_indices: [[1053, 487, 2405, 1333, 233, 2438, 1224, 673, 1792, 277, 103, 3282, 2895, 1778, 823, 14, 89, 3015, 3069, 1585]]
top_k_names: ['NPIPB15', 'AADAC', 'AMBP', 'LYPLA1', 'SLC12A8', 'RHBDL1', 'CYP3A5', 'ADGRF1', 'C1orf68', 'CYP21A2', 'IFI6', 'EPHA7', 'CP', 'FOLR3', 'CHP1', 'FKBP4', 'TRA2A', 'CERS4', 'CYP46A1', 'COX15']
contri_k_names: [['NPIPB15', 'AADAC', 'AMBP', 'LYPLA1', 'SLC12A8', 'RHBDL1', 'CYP3A5', 'ADGRF1', 'C1orf68', 'CYP21A2', 'IFI6

data: torch.Size([1, 3300])
rel_for_clas=None: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4941]],
       device='cuda:0', dtype=torch.float64)
top_k: torch.return_types.topk(
values=tensor([[0.0458, 0.0238, 0.0212, 0.0171, 0.0168, 0.0143, 0.0127, 0.0117, 0.0100,
         0.0096, 0.0082, 0.0081, 0.0077, 0.0074, 0.0074, 0.0071, 0.0069, 0.0069,
         0.0068, 0.0068]], dtype=torch.float64),
indices=tensor([[2802, 1155, 3223, 1426,   14, 1818, 3293, 2626,  989, 2666, 2408, 1585,
         2417,  556, 2207, 3015,  182,  826,  103,  914]]))
top_k_indices: [[2802, 1155, 3223, 1426, 14, 1818, 3293, 2626, 989, 2666, 2408, 1585, 2417, 556, 2207, 3015, 182, 826, 103, 914]]
top_k_names: ['PDE8A', 'RGN', 'RPS4Y1', 'WASHC3', 'FKBP4', 'ZNHIT1', 'LRRC23', 'PTMS', 'KLF6', 'HKDC1', 'AMDHD2', 'COX15', 'PRODH', 'ACAD10', 'MRPL19', 'CERS4', 'PAX7', 'RHOBTB1', 'IFI6', 'GDF15']
contri_k_names: [['PDE8A', 'RGN', 'RPS4Y1', 'WASHC3', 'FKBP4', 'ZNHIT1', 'LRRC23', 'PTMS', 'KLF6', 

...

top_k: torch.return_types.topk(
values=tensor([[0.0734, 0.0681, 0.0666, 0.0418, 0.0345, 0.0342, 0.0278, 0.0269, 0.0233,
         0.0223, 0.0183, 0.0177, 0.0174, 0.0169, 0.0149, 0.0147, 0.0146, 0.0140,
         0.0132, 0.0127]], dtype=torch.float64),
indices=tensor([[1781,  510, 1053, 1111, 3085,  884,   89,  673, 3017,  180, 3060, 1106,
         1184,  699, 2626,  879,  826, 1488,  277, 1881]]))
top_k_indices: [[1781, 510, 1053, 1111, 3085, 884, 89, 673, 3017, 180, 3060, 1106, 1184, 699, 2626, 879, 826, 1488, 277, 1881]]
top_k_names: ['KPNA3', 'ABCB7', 'NPIPB15', 'MAP3K1', 'NUBPL', 'KCNG2', 'TRA2A', 'ADGRF1', 'COL4A3BP', 'AUTS2', 'SERPINF1', 'SPI1', 'CFLAR', 'ADPRM', 'PTMS', 'RHOB', 'RHOBTB1', 'GATC', 'CYP21A2', 'ATP6V1F']
contri_k_names: [['KPNA3', 'ABCB7', 'NPIPB15', 'MAP3K1', 'NUBPL', 'KCNG2', 'TRA2A', 'ADGRF1', 'COL4A3BP', 'AUTS2', 'SERPINF1', 'SPI1', 'CFLAR', 'ADPRM', 'PTMS', 'RHOB', 'RHOBTB1', 'GATC', 'CYP21A2', 'ATP6V1F']]
rel_for_clas=None: tensor([[0.0000, 0.0000, 0.0000, 0.00

rel_for_clas=None: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9996]],
       device='cuda:0', dtype=torch.float64)
top_k: torch.return_types.topk(
values=tensor([[0.0697, 0.0602, 0.0555, 0.0364, 0.0298, 0.0268, 0.0254, 0.0222, 0.0211,
         0.0209, 0.0207, 0.0184, 0.0165, 0.0157, 0.0153, 0.0141, 0.0137, 0.0136,
         0.0130, 0.0127]], dtype=torch.float64),
indices=tensor([[2489, 2418, 3007,   14,  103, 1426, 1393, 3256,  987, 3017,  265, 2093,
         1422,  601,  180,  186, 2010,  951, 2691, 1224]]))
top_k_indices: [[2489, 2418, 3007, 14, 103, 1426, 1393, 3256, 987, 3017, 265, 2093, 1422, 601, 180, 186, 2010, 951, 2691, 1224]]
top_k_names: ['PTP4A1', 'NR1D2', 'CASP8', 'FKBP4', 'IFI6', 'WASHC3', 'NEFH', 'WARS2', 'NDUFA13', 'COL4A3BP', 'ACAT2', 'CYP39A1', 'TTC1', 'ACTA2', 'AUTS2', 'COMMD10', 'TMEM100', 'CHODL', 'MISP', 'CYP3A5']
contri_k_names: [['PTP4A1', 'NR1D2', 'CASP8', 'FKBP4', 'IFI6', 'WASHC3', 'NEFH', 'WARS2', 'NDUFA13', 'COL4A3BP', 'ACAT2',

rel_for_clas=None: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9997, 0.0000]],
       device='cuda:0', dtype=torch.float64)
top_k: torch.return_types.topk(
values=tensor([[0.3194, 0.0252, 0.0244, 0.0176, 0.0163, 0.0156, 0.0154, 0.0150, 0.0149,
         0.0141, 0.0136, 0.0136, 0.0126, 0.0123, 0.0113, 0.0106, 0.0102, 0.0099,
         0.0093, 0.0091]], dtype=torch.float64),
indices=tensor([[ 103,  386, 2418,  182, 1224, 2476, 1609, 2489, 1426,  877,   89, 2947,
          989,  713, 1184,  975, 1854,  335, 2276, 3223]]))
top_k_indices: [[103, 386, 2418, 182, 1224, 2476, 1609, 2489, 1426, 877, 89, 2947, 989, 713, 1184, 975, 1854, 335, 2276, 3223]]
top_k_names: ['IFI6', 'DNASE1L3', 'NR1D2', 'PAX7', 'CYP3A5', 'EGLN3', 'CTSG', 'PTP4A1', 'WASHC3', 'PF4', 'TRA2A', 'EPM2AIP1', 'KLF6', 'AFF4', 'CFLAR', 'KLF9', 'PEG10', 'TUBA4B', 'FMO2', 'RPS4Y1']
contri_k_names: [['IFI6', 'DNASE1L3', 'NR1D2', 'PAX7', 'CYP3A5', 'EGLN3', 'CTSG', 'PTP4A1', 'WASHC3', 'PF4', 'TRA2A', 'EPM2AIP1', 

rel_for_clas=None: tensor([[0.9990, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0', dtype=torch.float64)
top_k: torch.return_types.topk(
values=tensor([[0.0655, 0.0596, 0.0412, 0.0299, 0.0269, 0.0261, 0.0210, 0.0204, 0.0200,
         0.0195, 0.0189, 0.0174, 0.0172, 0.0169, 0.0149, 0.0148, 0.0147, 0.0144,
         0.0142, 0.0140]], dtype=torch.float64),
indices=tensor([[ 826, 2141,   17, 2139,  987, 2559, 2626, 2093,  673, 2010, 1818,  762,
          975,   89, 3223, 1402, 1155, 1171, 3007, 2412]]))
top_k_indices: [[826, 2141, 17, 2139, 987, 2559, 2626, 2093, 673, 2010, 1818, 762, 975, 89, 3223, 1402, 1155, 1171, 3007, 2412]]
top_k_names: ['RHOBTB1', 'TFAP2A', 'TFAM', 'C1orf56', 'NDUFA13', 'TYR', 'PTMS', 'CYP39A1', 'ADGRF1', 'TMEM100', 'ZNHIT1', 'AIRE', 'KLF9', 'TRA2A', 'RPS4Y1', 'PSPH', 'RGN', 'ZNF195', 'CASP8', 'RORA']
contri_k_names: [['RHOBTB1', 'TFAP2A', 'TFAM', 'C1orf56', 'NDUFA13', 'TYR', 'PTMS', 'CYP39A1', 'ADGRF1', 'TMEM100', 'ZNHIT1', 

prediction_value: tensor(3)
prediction_score: 25.36085456186699
data: torch.Size([1, 3300])
rel_for_clas=None: tensor([[0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0', dtype=torch.float64)
top_k: torch.return_types.topk(
values=tensor([[0.0665, 0.0573, 0.0367, 0.0319, 0.0314, 0.0300, 0.0300, 0.0229, 0.0210,
         0.0206, 0.0197, 0.0185, 0.0184, 0.0183, 0.0170, 0.0167, 0.0155, 0.0151,
         0.0136, 0.0135]], dtype=torch.float64),
indices=tensor([[1333, 2438, 2489,  914, 3293, 1270,  103, 2207, 2947,  989, 1390,  975,
         2409, 3223, 2418, 1808,  303, 1818, 2711,  535]]))
top_k_indices: [[1333, 2438, 2489, 914, 3293, 1270, 103, 2207, 2947, 989, 1390, 975, 2409, 3223, 2418, 1808, 303, 1818, 2711, 535]]
top_k_names: ['LYPLA1', 'RHBDL1', 'PTP4A1', 'GDF15', 'LRRC23', 'CFB', 'IFI6', 'MRPL19', 'EPM2AIP1', 'KLF6', 'HIRA', 'KLF9', 'AMFR', 'RPS4Y1', 'NR1D2', 'OLFML3', 'TAF4B', 'ZNHIT1', 'CPVL', 'ABHD2']
contri_k_names: [['LYPLA1', 'RHBD